In [ ]:
import json
import random
import os
from pathlib import Path
from datasets import Dataset 
from sentence_transformers import (
    SentenceTransformer,
    losses,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)

ROOT = Path(".")

TRAIN_DATA_PATH = ROOT / "data" / "retriever_train.jsonl"
VAL_DATA_PATH   = ROOT / "data" / "retriever_val.jsonl"

LAW_DB_PATH = ROOT / "data" / "law_db.json"
OUTPUT_DIR  = ROOT / "models" / "kure-law-retriever"

def load_law_db():
    if not LAW_DB_PATH.exists():
        raise FileNotFoundError(f"법령 DB 파일이 없습니다: {LAW_DB_PATH}")
    with LAW_DB_PATH.open("r", encoding="utf-8") as f:
        law_list = json.load(f)
    return { int(r["id"]): r["law_text"] for r in law_list }

def build_dataset_from_file(file_path, law_db):
    anchors = []
    positives = []
    
    print(f"데이터 로드 중: {file_path}")
    if not file_path.exists():
        raise FileNotFoundError(f"파일이 없습니다: {file_path}")

    with file_path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line: continue
            
            ex = json.loads(line)
            clause = ex.get("input", "")
            gt_ids = ex.get("ground_truth_ids", [])
            
            if not clause or not gt_ids: continue
            
            # 정답 중 하나 랜덤 선택
            pos_id = random.choice(gt_ids)
            pos_text = law_db.get(int(pos_id))
            
            if not pos_text: continue
            
            anchors.append(clause)
            positives.append(pos_text)

    print(f" - 데이터 개수: {len(anchors)}")
    
    # HuggingFace Dataset 객체 반환
    return Dataset.from_dict({
        "anchor": anchors,
        "positive": positives
    })

def main():
    #  법령 DB 로드
    id2text = load_law_db()
    
    #  Train / Val 데이터셋 로드
    print("\n[Train Dataset]")
    train_dataset = build_dataset_from_file(TRAIN_DATA_PATH, id2text)
    
    print("\n[Val Dataset]")
    eval_dataset = build_dataset_from_file(VAL_DATA_PATH, id2text)

    #  모델 로드
    model = SentenceTransformer("nlpai-lab/KURE-v1")

    train_loss = losses.MultipleNegativesRankingLoss(model)

    args = SentenceTransformerTrainingArguments(
        output_dir=str(OUTPUT_DIR),
        num_train_epochs=1,             
        per_device_train_batch_size=2,  
        per_device_eval_batch_size=2,  
        gradient_accumulation_steps=8, 
        learning_rate=2e-5,            
        warmup_steps=100,               
        fp16=True,                   
        logging_steps=10,             
        save_strategy="epoch",          
        
        eval_strategy="epoch",          
    )

    #  Trainer 실행
    trainer = SentenceTransformerTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,      
        loss=train_loss,
    )

    print("\n학습 시작...")
    trainer.train()
    
    # 저장
    model.save_pretrained(str(OUTPUT_DIR))
    print(f"\n학습 및 저장 완료: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

/home/ally010314/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[Train Dataset]
📂 데이터 로드 중: data/retriever_train.jsonl
 - 데이터 개수: 2430

[Val Dataset]
📂 데이터 로드 중: data/retriever_val.jsonl
 - 데이터 개수: 270



🔥 학습 시작...


Epoch,Training Loss,Validation Loss
1,0.483800,0.368109



✅ 학습 및 저장 완료: models/kure-law-retriever-final


In [ ]:
import os
import faiss
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

JSON_PATH = "data/law_db.json" 
INDEX_DIR = "rag_index"
EMBEDDING_MODEL = os.environ.get("EMB_MODEL", "nlpai-lab/KURE-v1")
#EMBEDDING_MODEL = "models/kure-law-retriever"

BATCH = 128


def get_model(name: str):
    print(f"[Embedding] loading: {name}")
    model = SentenceTransformer(name)
    return model


def e5_encode_query(text: str) -> str:
    return "passage: " + text.strip()


def main():
    os.makedirs(INDEX_DIR, exist_ok=True)
    df = pd.read_json(JSON_PATH)
    assert "law_text" in df.columns, "JSON에 'law_text' 필드가 필요합니다."

    df = df.drop_duplicates(subset=["law_text"]).reset_index(drop=True)
    print(f"[Data] unique rows: {len(df)}")

    model = get_model(EMBEDDING_MODEL)
    texts = [e5_encode_query(t) for t in df["law_text"].tolist()]

    # 배치 임베딩
    all_vecs = []
    for i in tqdm(range(0, len(texts), BATCH), desc="Embedding"):
        batch = texts[i:i + BATCH]
        vecs = model.encode(batch, normalize_embeddings=True, convert_to_numpy=True)
        all_vecs.append(vecs)

    X = np.vstack(all_vecs).astype("float32")
    dim = X.shape[1]

    # FAISS Index
    index = faiss.IndexFlatIP(dim)
    index.add(X)
    faiss.write_index(index, os.path.join(INDEX_DIR, "faiss.index"))

    #메타데이터 저장 (id, law_text만 유지)
    meta = []
    for _, row in df.iterrows():
        meta.append({
            "id": int(row["id"]),
            "file_name": None,        
            "clauseField": None,
            "law_text": row["law_text"],
        })

    with open(os.path.join(INDEX_DIR, "meta.pkl"), "wb") as f:
        pickle.dump(meta, f)

    with open(os.path.join(INDEX_DIR, "dim.txt"), "w", encoding="utf-8") as f:
        f.write(str(dim))

    print(f"vectors={len(df)}, dim={dim}")
    print(f"- FAISS : {os.path.join(INDEX_DIR, 'faiss.index')}")
    print(f"- META  : {os.path.join(INDEX_DIR, 'meta.pkl')}")
    print(f"- DIM   : {os.path.join(INDEX_DIR, 'dim.txt')}")


if __name__ == "__main__":
    main()


[Data] unique rows: 38
[Embedding] loading: nlpai-lab/KURE-v1


Embedding: 100%|██████████| 1/1 [00:03<00:00,  3.71s/it]

vectors=38, dim=1024
- FAISS : rag_index/faiss.index
- META  : rag_index/meta.pkl
- DIM   : rag_index/dim.txt


In [15]:
import json
import os
import pickle
import numpy as np
import faiss
import math
from pathlib import Path
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

ROOT = Path(".")
DATA_PATH = ROOT / "data" / "retriever_val.jsonl"

BASE_MODEL_NAME = "nlpai-lab/KURE-v1"

INDEX_PATH = ROOT / "rag_index" / "faiss.index"
META_PATH  = ROOT / "rag_index" / "meta.pkl"

def load_resources():
    print(f"Base 모델 로드 중: {BASE_MODEL_NAME}")
    embedder = SentenceTransformer(BASE_MODEL_NAME)
    
    print(f"인덱스 로드 중: {INDEX_PATH}")
    if not INDEX_PATH.exists():
        raise FileNotFoundError(f"인덱스 파일이 없습니다: {INDEX_PATH}")
        
    index = faiss.read_index(str(INDEX_PATH))
    
    with open(META_PATH, "rb") as f:
        meta = pickle.load(f)
        
    return embedder, index, meta

def calculate_metrics(hits, gt_ids):
    # Recall
    r1  = 1.0 if any(h in gt_ids for h in hits[:1]) else 0.0
    r3  = 1.0 if any(h in gt_ids for h in hits[:3]) else 0.0
    r5  = 1.0 if any(h in gt_ids for h in hits[:5]) else 0.0
    r10 = 1.0 if any(h in gt_ids for h in hits[:10]) else 0.0
    
    # MRR
    mrr = 0.0
    for rank, h in enumerate(hits, 1):
        if h in gt_ids:
            mrr = 1.0 / rank
            break
            
    # nDCG@5
    dcg = 0.0
    for rank, h in enumerate(hits[:5], 1):
        if h in gt_ids:
            dcg = 1.0 / math.log2(rank + 1)
            break
            
    return r1, r3, r5, r10, mrr, dcg

def main():
    # 1. 리소스 로드
    embedder, index, meta = load_resources()
    
    # 2. 데이터 로드
    print(f"평가 데이터 로드: {DATA_PATH}")
    with open(DATA_PATH, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f if line.strip()]
        
    print(f"\n[Case 1: Base Model] 평가 시작 (총 {len(data)}개)...")
    
    stats = {"r1": 0.0, "r3": 0.0, "r5": 0.0, "r10": 0.0, "mrr": 0.0, "ndcg5": 0.0}
    cnt = 0
    
    for ex in tqdm(data):
        clause = ex.get("input", "")
        gt_ids = set(ex.get("ground_truth_ids", []))
        
        if not gt_ids: continue
        
        query = clause
        
        vec = embedder.encode([query], normalize_embeddings=True)
        _, I = index.search(vec.astype("float32"), 10)
        
        pred_ids = []
        for idx in I[0]:
            if idx == -1: continue
            pred_ids.append(meta[int(idx)]["id"])
            
        r1, r3, r5, r10, mrr, ndcg = calculate_metrics(pred_ids, gt_ids)
        
        stats["r1"] += r1
        stats["r3"] += r3
        stats["r5"] += r5
        stats["r10"] += r10
        stats["mrr"] += mrr
        stats["ndcg5"] += ndcg
        cnt += 1
        
    if cnt == 0:
        print("평가할 데이터가 없습니다.")
        return

    for k in stats: stats[k] /= cnt
    
    print("\n" + "="*40)
    print(f"[Case 1 결과] KURE-v1 Base Model")
    print("="*40)
    print(f"Recall@1  : {stats['r1']:.4f}")
    print(f"Recall@3  : {stats['r3']:.4f}")
    print(f"Recall@5  : {stats['r5']:.4f}")
    print(f"Recall@10 : {stats['r10']:.4f}")
    print(f"MRR       : {stats['mrr']:.4f}")
    print(f"nDCG@5    : {stats['ndcg5']:.4f}")
    print("="*40)

if __name__ == "__main__":
    main()

Base 모델 로드 중: nlpai-lab/KURE-v1
인덱스 로드 중: rag_index/faiss.index
평가 데이터 로드: data/retriever_val.jsonl

[Case 1: Base Model] 평가 시작 (총 270개)...


100%|██████████| 270/270 [00:17<00:00, 15.22it/s]



[Case 1 결과] KURE-v1 Base Model
Recall@1  : 0.0889
Recall@3  : 0.2111
Recall@5  : 0.2926
Recall@10 : 0.4556
MRR       : 0.1847
nDCG@5    : 0.1964


In [16]:
import json
import os
import pickle
import numpy as np
import faiss
import math
from pathlib import Path
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

ROOT = Path(".")
DATA_PATH = ROOT / "data" / "retriever_val.jsonl" 


MODEL_PATH = ROOT / "models" / "kure-law-retriever"
INDEX_PATH = ROOT / "rag_index_retriever" / "faiss.index" 
META_PATH  = ROOT / "rag_index_retriever" / "meta.pkl"    

def load_resources():
    print(f" Fine-tuned 모델 로드: {MODEL_PATH}")
    embedder = SentenceTransformer(str(MODEL_PATH))
    
    print(f" FAISS 인덱스 로드: {INDEX_PATH}")
    if not INDEX_PATH.exists():
        raise FileNotFoundError(f"인덱스 파일이 없습니다: {INDEX_PATH}")
        
    index = faiss.read_index(str(INDEX_PATH))
    with open(META_PATH, "rb") as f:
        meta = pickle.load(f)
    return embedder, index, meta

def parse_reason_from_answer(answer_line: str) -> str:
    """
    sft_answer 문자열: "분야: 3 / 불공정여부: 불리 / 근거: ......"
    여기서 '근거: ' 뒤의 텍스트만 추출함.
    """
    if not answer_line:
        return ""
        
    parts = [p.strip() for p in answer_line.split("/") if p.strip()]
    
    for p in parts:
        if p.startswith("근거:"):
            return p.replace("근거:", "").strip()
            
    return ""

def calculate_metrics(hits, gt_ids):
    # Recall
    r1  = 1.0 if any(h in gt_ids for h in hits[:1]) else 0.0
    r3  = 1.0 if any(h in gt_ids for h in hits[:3]) else 0.0
    r5  = 1.0 if any(h in gt_ids for h in hits[:5]) else 0.0
    r10 = 1.0 if any(h in gt_ids for h in hits[:10]) else 0.0
    
    # MRR
    mrr = 0.0
    for rank, h in enumerate(hits, 1):
        if h in gt_ids:
            mrr = 1.0 / rank
            break
    # nDCG@5
    dcg = 0.0
    for rank, h in enumerate(hits[:5], 1):
        if h in gt_ids:
            dcg = 1.0 / math.log2(rank + 1)
            break
    return r1, r3, r5, r10, mrr, dcg 

def evaluate(name, data, embedder, index, meta, mode):
    print(f"\n [{name}] 평가 중...")
    stats = {"r1": 0.0, "r3": 0.0, "r5": 0.0, "r10": 0.0, "mrr": 0.0, "ndcg5": 0.0}
    cnt = 0
    
    for ex in tqdm(data):
        clause = ex.get("input", "")
        gt_ids = set(ex.get("ground_truth_ids", []))
        if not gt_ids: continue
        
        if mode == "reason": 
            sft_ans = ex.get("sft_answer", "")
            
            reason = parse_reason_from_answer(sft_ans)
            
            if reason and reason != "해당 없음":
                query = f"{clause}\n\n판단근거: {reason}"
            else:
                query = clause
        else:
            query = clause
            
        vec = embedder.encode([query], normalize_embeddings=True)
        _, I = index.search(vec.astype("float32"), 10)
        
        pred_ids = []
        for idx in I[0]:
            if idx == -1: continue
            pred_ids.append(meta[int(idx)]["id"])
            
        r1, r3, r5, r10, mrr, ndcg = calculate_metrics(pred_ids, gt_ids)
        stats["r1"] += r1
        stats["r3"] += r3
        stats["r5"] += r5
        stats["r10"] += r10
        stats["mrr"] += mrr
        stats["ndcg5"] += ndcg
        cnt += 1
        
    for k in stats: stats[k] /= cnt
    return stats

def main():
    embedder, index, meta = load_resources()
    
    print(f" 데이터 로드: {DATA_PATH}")
    with open(DATA_PATH, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f if line.strip()]
        
    res_case2 = evaluate("Case 2: Only Clause (Fine-tuned)", data, embedder, index, meta, mode="clause")
    
    res_case1 = evaluate("Case 3: Clause + Reason (Fine-tuned)", data, embedder, index, meta, mode="reason")
    
    print("\n" + "="*60)
    print(f"{'Metric':<10} | {'Case 2 (Only Clause)':<22} | {'Case 3 (+Reason)':<22}")
    print("-" * 60)
    for k in ["r1", "r3", "r5", "r10", "mrr", "ndcg5"]:
        diff = res_case1[k] - res_case2[k]
        print(f"{k:<10} | {res_case2[k]:.4f}                 | {res_case1[k]:.4f} ({diff:+.4f})")
    print("="*60)

if __name__ == "__main__":
    main()

 Fine-tuned 모델 로드: models/kure-law-retriever
 FAISS 인덱스 로드: rag_index_retriever/faiss.index
 데이터 로드: data/retriever_val.jsonl

 [Case 2: Only Clause (Fine-tuned)] 평가 중...


100%|██████████| 270/270 [04:08<00:00,  1.09it/s]



 [Case 3: Clause + Reason (Fine-tuned)] 평가 중...


100%|██████████| 270/270 [07:46<00:00,  1.73s/it]


Metric     | Case 2 (Only Clause)   | Case 3 (+Reason)      
------------------------------------------------------------
r1         | 0.3778                 | 0.3926 (+0.0148)
r3         | 0.6185                 | 0.6148 (-0.0037)
r5         | 0.6926                 | 0.7148 (+0.0222)
r10        | 0.8370                 | 0.8519 (+0.0148)
mrr        | 0.5235                 | 0.5332 (+0.0097)
ndcg5      | 0.5519                 | 0.5647 (+0.0128)
